<a href="https://colab.research.google.com/github/katduecker/dnn_osci/blob/main/Thesis_Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Characterise a single-neuron dynamical system


# one neuron: describe dynamical system as a function of input
# show that it is easily entrained
# pulses build uo earlier for neurons with strong inputs

# "population rate model"

# H~R system
# justify max(Z) scaling with homeostatic processes


# timing/params: Abbot & DAyan
# timing calcium dependent potassium channels

In [ ]:
# 2) NN: 1 layer -> read out can be entrained to alpha

# With set bias and slope to ensure sparsity constraints & orthogonality, high activation
# Also show optimized sparsity constraints
# Show how alpha reflects a duty cycle

# temporal code

In [ ]:
# 3) 2 layer NN

# relationship between phase difference layer 1 & 2 and temporal codeness
# pipelining?